The database records the emergency 911 calls over an interval of time.
the features are as follows:

These two features represent the location as identified by the Opearator
lat : String variable, Latitude
lng: String variable, Longitude

desc: String variable, Description of the Emergency Call, reason and nature of emergency
zip: String variable, Zipcode of the reporter as provided by the caller
title: String variable, Title
timeStamp: String variable, YYYY-MM-DD HH:MM:SS
twp: String variable, Township
addr: String variable, Address
e: String variable, Dummy variable (always 1)

In [ ]:
# Hello this is my first project on kaggle. I would appreciate feedback so much
# Thank you :)

In [ ]:
# Reading the data
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Loading the needed liberaries needed for analysing
import numpy as np # linear algebra
import pandas as pd # for Reading, Analysing and Manipulating Data
import matplotlib.pyplot as plt # Visualizing Data
import seaborn as sns # Visualizing Data
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
# Reading the data
df = pd.read_csv('/kaggle/input/911-calls/911.csv') 

In [ ]:
# getting a glance of what the data and features looks like though viewing the very first raws
df.head()

# What Features are available in the dataset? and what are their types?

In [ ]:
# Understanding the data even more by knowing the types of the features and getting an idea of the null
# values..
# It is apparent that the zip code of the instance is the least, with the most non-null values, this gives us a
# a good view of the data.
df.info()

# How many rows and columns does the dataset have?

In [ ]:
df.shape

# Which features are categorical?

In [ ]:
df.select_dtypes('object')

# Which features are numerical?

In [ ]:
df.select_dtypes(['float64','int64'])

# Which features contain blank, null or empty values?

In [ ]:
df.isnull().sum()

In [ ]:
# It is appearant that the 'zip' feature has the most null values

# How many zip codes does the dataset have?

In [ ]:
df['zip'].nunique()

# What are the top 5 zip codes that has the most emergency calls?

In [ ]:
df['zip'].value_counts().head()

# What are the top 5 townships (twp) for 911 calls?

In [ ]:
df['twp'].value_counts().head()

# What the most reported emergencies?

In [ ]:
# This way is not effective as there are, for example, different types of traffic or EMS
df['title'].value_counts()

In [ ]:
# To get the exact reason for the emergency call, we need to reason extracted from the title feature
# Creating a new feature called reason using apply method
# Applying the apply method on a feature, maps the function on the column raw by raw..
df['reason'] = df['title'].apply(lambda x : x.split(':')[0])

In [ ]:
# Examining the new column within the table of data.
# The new 'reason' feature is of the type String
df.head()

In [ ]:
# We can know the most reporting reason
df['reason'].value_counts()

In [ ]:
# We can import the Warnings liberary, to hid the warnings that happen with seaborn visuals
import warnings
warnings.filterwarnings('ignore')
# Visualising the count of the emergencies call by the reason of call
sns.countplot(df['reason'],data=df)

# What are the most days of the month of reporting traffic?

In [ ]:
# To report this, we need to analyse the months againts the number of traffic calls..
type(df['timeStamp'][0]) # the timestamp feature is String, which is of not much help to analysis.

In [ ]:
# Hence, we can turn the timestamp from a string into a timestamp object for more reliable view.
# Overwriting the 'timestamp' feature applying the apply method using the pd.date_time method 
# that changes A time stamp feature of a type
# string into an actual timestamp object.
df['timeStamp'] = df['timeStamp'].apply(lambda x : pd.to_datetime(x))

In [ ]:
# Verifying the change in the timestamp object that is has been already changed into 'datetime64[ns]'
df.info()

In [ ]:
# Creating new features from the timestamp feature of the ease of analysing and visualization
# We can create a new feature of the month of each emergency call, 
# the new feature is of the type int
df['month'] = df['timeStamp'].apply(lambda x : x.month)
# We can now try to answer the question 'What are the most days of the month of reporting traffic?'

In [ ]:
# What are the most days of the month of reporting traffic?
# January
sns.countplot(x=df['month'],data=df[df['reason']=='Traffic'])

# What are the number of calls for each month?

In [ ]:
# It is apparent that the number calls decreases by the end of the year.
sns.countplot(x=df['month'],data=df)

In [ ]:
sns.countplot(x=df['month'],data=df,hue=df['reason'])

# What are the most hours of the day of reporting EMS?

In [ ]:
# We can create a new feature of the hour of each emergency call, 
# the new feature is of the type int
df['hour'] = df['timeStamp'].apply(lambda x : x.hour)
sns.countplot(x=df['hour'],data=df[df['reason']=='EMS'])
# It is apparent that the the most hours of the day that EMS happen are the afternoon hours.

# What are the most days of the week of reporting EMS?

In [ ]:
# We can create a new feature of the day of the week of each emergency call,
# Using apply and the attribute of datetime objects '.dayofweek' which return a numerical value of the day
# as such:
numDays_vs_weekDay = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
df['day of the week'] = df['timeStamp'].apply(lambda x : x.dayofweek)
# we can label the days of the week instead of the numbers but with string represenation of the day.
df['day of the week'] = df['day of the week'].map(numDays_vs_weekDay)
# Now we can answer of What are the most days of the week of reporting EMS?
sns.countplot(x=df['day of the week'],data=df[df['reason']=='EMS'])
# the results are pretty similar that it appears there is not significant increase of EMS calls in
# a specific day of the week.

In [ ]:
# We can still visualize the count of all calls with respect to their reason across the whole interval
sns.countplot(x=df['day of the week'],data=df,hue=df['reason'])
# Relocate the legend box outside the graph for a better view
plt.legend(bbox_to_anchor=(1, 1))
# There is not increase significant of any specific call on any given day of the week
# However, there is a decrease in the traffic call on Sunday

In [ ]:
# We can look even more closer into the day of the week though analysing the hours of each day.
# df.groupby(['day of the week','hour']).count() # a groupby which matches the day of the week with each hour of it
# We can use .unstack() to rearrange the indexes which matches the day with the hour
# df.groupby(['day of the week','hour']).count()['reason'] # we can have the only feature to look at as the reason
day_vs_hour_counted_by_reason= df.groupby(['day of the week','hour']).count()['reason'].unstack()
day_vs_hour_counted_by_reason

In [ ]:
# We want to answer undertand from a heatmap is the correlation between the hour of the day 
# and hour of days
# Increasing the size of the figure
plt.figure(figsize=(12,8))
# Creating the heatmap using seaborn
sns.heatmap(day_vs_hour_counted_by_reason)
# It is apparent that the late hours of the night has the least emergency calls
# However on the weekends, there is a more emergency calls after midnight.